In [1]:
import pandas as pd
df=pd.read_csv("D:\\UB\\Dicoding\\GitHub\\Capstone_Akmal_Andri_Zahran\\recipes_new.csv")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52785 entries, 0 to 52784
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   RecipeId                    52785 non-null  int64  
 1   Name                        52785 non-null  object 
 2   AuthorId                    52785 non-null  int64  
 3   AuthorName                  52785 non-null  object 
 4   CookTime                    52785 non-null  int64  
 5   PrepTime                    52785 non-null  int64  
 6   TotalTime                   52785 non-null  int64  
 7   DatePublished               52785 non-null  object 
 8   Description                 52785 non-null  object 
 9   Images                      52785 non-null  object 
 10  RecipeCategory              52785 non-null  object 
 11  Keywords                    52785 non-null  object 
 12  RecipeIngredientQuantities  52785 non-null  object 
 13  RecipeIngredientParts       527

In [3]:
import pandas as pd  
import numpy as np  
from sklearn.preprocessing import MultiLabelBinarizer  
from sklearn.cluster import KMeans  
from ast import literal_eval  


In [4]:
class MealPlanGenerator:  
    def __init__(self, df):  
        self.df = df  
        self.preprocess_data()  
        self.cluster_recipes()  
    
    def preprocess_data(self):  
        # Preprocessing untuk kategorisasi keywords  
        def parse_keywords(keywords):  
            try:  
                # Konversi string ke list  
                if isinstance(keywords, str):  
                    return [k.strip() for k in keywords.split(',')]  
                return keywords  
            except:  
                return []  
        
        # Kategorisasi waktu makan  
        meal_time_keywords = {  
            'Breakfast': ['Breakfast', 'Brunch', 'Oatmeal', 'Eggs'],  
            'Lunch': ['Lunch/Snacks', 'Salad', 'Sandwich'],  
            'Dinner': ['Dinner', 'Meat', 'Chicken', 'Fish', 'Steak', 'Stir Fry']  
        }  
        
        # Parse keywords  
        self.df['ParsedKeywords'] = self.df['Keywords'].apply(parse_keywords)  
        
        # Kategorisasi meal type  
        def categorize_meal_type(keywords):  
            for meal, meal_keywords in meal_time_keywords.items():  
                if any(kw in keywords for kw in meal_keywords):  
                    return meal  
            return 'Other'  
        
        self.df['MealType'] = self.df['ParsedKeywords'].apply(categorize_meal_type)  
        
        # Multi-label encoding untuk keywords  
        mlb = MultiLabelBinarizer()  
        keyword_encoded = mlb.fit_transform(self.df['ParsedKeywords'])  
        self.keyword_columns = mlb.classes_  
    
    def cluster_recipes(self):  
        # Fitur untuk clustering  
        features = self.df[['Calories', 'ProteinContent', 'CarbohydrateContent']]  
        
        # Normalisasi fitur  
        from sklearn.preprocessing import StandardScaler  
        scaler = StandardScaler()  
        features_scaled = scaler.fit_transform(features)  
        
        # Clustering menggunakan KMeans  
        self.kmeans = KMeans(n_clusters=5, random_state=42)  
        self.df['Cluster'] = self.kmeans.fit_predict(features_scaled)  
    
    def generate_meal_plans(self, total_calories, max_plans=4):  
        meal_plans = []  
        
        # Filter kategori makanan  
        breakfast_df = self.df[self.df['MealType'] == 'Breakfast']  
        lunch_df = self.df[self.df['MealType'] == 'Lunch']  
        dinner_df = self.df[self.df['MealType'] == 'Dinner']  
        
        # Generate meal plans  
        while len(meal_plans) < max_plans:  
            # Sampling dengan mempertimbangkan cluster  
            breakfast = breakfast_df.sample(1).iloc[0]  
            lunch = lunch_df.sample(1).iloc[0]  
            dinner = dinner_df.sample(1).iloc[0]  
            
            # Hitung total kalori  
            total_plan_calories = (  
                breakfast['Calories'] +   
                lunch['Calories'] +   
                dinner['Calories']  
            )  
            
            # Validasi total kalori  
            if total_plan_calories <= total_calories:  
                meal_plan = {  
                    'Breakfast': {  
                        'Name': breakfast['Name'],  
                        'Calories': breakfast['Calories'],  
                        'Keywords': breakfast['ParsedKeywords']  
                    },  
                    'Lunch': {  
                        'Name': lunch['Name'],  
                        'Calories': lunch['Calories'],  
                        'Keywords': lunch['ParsedKeywords']  
                    },  
                    'Dinner': {  
                        'Name': dinner['Name'],  
                        'Calories': dinner['Calories'],  
                        'Keywords': dinner['ParsedKeywords']  
                    },  
                    'Total Calories': total_plan_calories  
                }  
                meal_plans.append(meal_plan)  
        
        return meal_plans  
    
    def display_meal_plans(self, meal_plans):  
        for i, plan in enumerate(meal_plans, 1):  
            print(f"Meal {i}")  
            for meal_type, meal_info in plan.items():  
                if meal_type != 'Total Calories':  
                    print(f"{meal_type}: {meal_info['Name']} ({meal_info['Calories']} calories)")  
                    print(f"Keywords: {', '.join(meal_info['Keywords'])}")  
            print(f"Total Calories: {plan['Total Calories']}\n")  


In [5]:
# Inisialisasi generator  
generator = MealPlanGenerator(df)  

In [31]:
# Generate meal plan dengan total kalori 1000  
total_calories = 3000 
meal_plans = generator.generate_meal_plans(total_calories)  

# Tampilkan meal plan  
generator.display_meal_plans(meal_plans)  

Meal 1
Breakfast: Shrimp and Tomato Quiche (344 calories)
Keywords: New Zealand, Australian, Brunch, < 60 Mins
Lunch: The Works -  Bonza Aussie Burger and Chips! (684 calories)
Keywords: Lunch/Snacks, Cheese, Tropical Fruits, Fruit, Meat, Australian, Toddler Friendly, Kid Friendly, Summer, Savory, < 60 Mins
Dinner: Broiled Steak with Shallot Butter (798 calories)
Keywords: Steak, Grains, Vegetable, Meat, Scottish, European, Free Of..., Broil/Grill, < 30 Mins, Oven, Beginner Cook, Easy
Total Calories: 1826

Meal 2
Breakfast: Potage Parmentier (Potato &amp; Leek Soup) – Julia Child (181 calories)
Keywords: Vegetable, European, Low Protein, Low Cholesterol, Healthy, Kosher, Brunch, < 4 Hours
Lunch: Spam, Broccoli & Cheese Wreath (388 calories)
Keywords: Lunch/Snacks, Cheese, < 60 Mins, Easy
Dinner: Pork Chops and Dressing (396 calories)
Keywords: Meat, < 4 Hours, Easy
Total Calories: 965

Meal 3
Breakfast: Southwestern Green Chili Quiche (386 calories)
Keywords: Cheese, Southwestern U.S.,

In [14]:
import joblib  
# Simpan model dan generator  
joblib.dump(generator.kmeans, 'kmeans_model.joblib')  
joblib.dump(generator, 'meal_plan_generator.joblib')

['meal_plan_generator.joblib']